# Importações

In [1]:
# Preparando ambiente (importando bibliotecas e downloads...)

!pip install nltk
import nltk
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')
import re
import pandas as pd 
import numpy as np
import spacy
nlp = spacy.load('pt')

/bin/bash: pip: comando não encontrado


[nltk_data] Downloading package rslp to /home/karine/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/karine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Carregamento e tratamento dos léxicos e datasets necessários

In [2]:
dicionario_lexico = {}
sentilexpt = open('lexicos/SentiLex-lem-PT02.txt','r', encoding='utf8')
ontolp = open('lexicos/lexico_v3.0.txt','r', encoding='utf8')

In [3]:
for i in sentilexpt:
    pos_ponto = i.find('.')
    palavra = (i[:pos_ponto])
    pol_pos = i.find('POL')
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')
    dicionario_lexico[palavra] = polaridade

In [4]:
for i in ontolp:
    split_dic = i.split(',')
    palavra = split_dic[0]
    if palavra not in dicionario_lexico:
        polaridade = split_dic[2]
        dicionario_lexico[palavra] = polaridade        

In [5]:
dic_sentimentos = {}
sentidic = open('Dicionarios/Plutchik/dic_sentimento_4_niveis_balanceado.txt','r', encoding='utf8')

In [6]:
for i in sentidic:
    split_dic = i.split('\t')
    dic_sentimentos[split_dic[0]] = split_dic[1][:-1]

In [7]:
degree_words = open('lexicos/degree-words.txt','r', encoding='utf8')
degree_words_set = {}

In [8]:
for i in degree_words:
    split_dic = i.split(';')
    degree_words_set[split_dic[0]] = split_dic[1][:-1]

In [9]:
sentilexpt.close()
ontolp.close()
degree_words.close()
sentidic.close()

In [10]:
liwc = pd.read_csv("lexicos/LIWC2007.txt",sep="\t",encoding="ISO-8859-1", names=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

In [11]:
liwc_set = {}
def cria_dic_liwc(x):
    lista = []
    if 126 in list(x):
        liwc_set[x.name] = 'pos'
    elif 127 in list(x):
        liwc_set[x.name] = 'neg'

In [12]:
liwc.apply(lambda x: cria_dic_liwc(x), axis=1)

a          None
aba        None
abafa      None
abafad*    None
abafada    None
           ... 
último     None
último*    None
út*        None
úteis      None
útil       None
Length: 127161, dtype: object

## Definições

In [13]:
dicionario_def = dic_sentimentos
#sentimentos = ['alegria','medo','nojo','raiva','surpresa', 'tristeza']
#pontuacao_sent = {'alegria':0, 'medo':0, 'nojo':0, 'raiva':0, 'surpresa':0, 'tristeza':0}

sentimentos = ['alegria','antecipação','confiança','medo','nojo', 'raiva', 'surpresa', 'tristeza']
pontuacao_sent = {'alegria':0, 'antecipação':0, 'confiança':0, 'medo':0, 'nojo':0, 'raiva':0, 'surpresa':0, 'tristeza':0}

# Sentimentos por transação

In [14]:
def taxa_sent_por_frase(frase):
    
    global pontuacao_sent
    
    frase = nlp(frase)
    degree_words_head = []
    degree_words_aux = []
    score_frase = pontuacao_sent.copy()
    
    for token in frase:
        if(token.text in degree_words_set):
            degree_words_head.append(token.head.text)
            degree_words_aux.append(int(degree_words_set.get(token.text)))
            
    for token in frase:
        if(token.text in dic_sentimentos):
            dic = dicionario_def.get(token.text)
            neg = 0
            if token.head.text in degree_words_head:                
                neg = int(degree_words_aux[degree_words_head.index(token.head.text)])
            score_frase[dic] = int(score_frase.get(dic)) + 1 + neg

    score_resultado = ('neutro', 0)
    for sentimento, valor in score_frase.items():
        s, v = score_resultado
        if abs(valor) > v:
            score_resultado = (sentimento, valor)
        
    s, v = score_resultado
    return s

In [15]:
def sentimento_transacao(transacao):
    lista_sentimentos = []
    agentes = []
    sent_max = ''
    sent_v = 0
    for frase in transacao:
        res = frase.split('\t')
        lista_sentimentos.append(taxa_sent_por_frase(res[1]))
        agentes.append(res[0])
    for item in sentimentos:
        v_sent_atual = lista_sentimentos.count(item)
        if v_sent_atual > sent_v:
            sent_v = v_sent_atual
            sent_max = item
    v_agentes = pd.DataFrame(agentes)[0].value_counts().index.tolist()
    if len(v_agentes) > 2:
        print('ERRO contagem de agentes')
    return sent_max, v_agentes

In [16]:
registros = open('logs/Logs_transacoes/Registros-Parcial.txt', 'r', encoding='utf8')
df_sentimentos_classificados = pd.DataFrame(columns=['Partida', 'Rodada', 'Etapa', 'Emp', 'Agro'])
for i in registros:
    aux = str('logs/Logs_transacoes/Transacoes_Corrigidas_Concluidas/' + i[:-1] + ".txt")
    log_transacao = open(aux, 'r', encoding='utf8')
    df = i[:-1].split('_')
    sent, agentes = sentimento_transacao(log_transacao)
    if "Emp" in agentes[0]:
        a = agentes[0]
        b = agentes[1]
    else:
        a = agentes[1]
        b = agentes[0]
    df_aux = pd.DataFrame([[df[0][-1:], df[1][-1:], df[2][-1:], a, b, sent]], columns=['Partida', 'Rodada', 'Etapa', 'Emp', 'Agro', 'Sentimento'])
    df_sentimentos_classificados = df_sentimentos_classificados.append(df_aux, ignore_index=True)
df_sentimentos_classificados.to_csv('sentimentos_classificados.txt')  